In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.metrics import f1_score

In [2]:
### Constants
seed = 69
cv_size = .3

train = pd.read_csv("data/Train.csv", encoding="ISO-8859-1")
test = pd.read_csv("data/Test.csv", encoding="ISO-8859-1")

X = train.loc[:, train.columns != 'revenue']
y = train["revenue"]

# encode string input values as integers
features = []
encode_cols = ["browser", "operatingSystem", "deviceCategory", "continent", 
              "subContinent", "country", "region", "metro", "city", "medium"]

other_cols = ["visitNumber", "isMobile", "hits", "pageviews", "bounces", "newVisits"]

# encode string input values as integers
encoded_x = None
for i in encode_cols:
	label_encoder = LabelEncoder()
	feature = label_encoder.fit_transform(X[i].fillna('0'))
	feature = feature.reshape(X.shape[0], 1)
	onehot_encoder = OneHotEncoder(sparse=False)
	feature = onehot_encoder.fit_transform(feature)
	if encoded_x is None:
		encoded_x = feature
	else:
		encoded_x = np.concatenate((encoded_x, feature), axis=1)
# print("X shape: ", encoded_x.shape)

X = np.concatenate((X[other_cols].fillna(0), encoded_x), axis=1)
y = Binarizer().transform(y.values.reshape(-1, 1))
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=cv_size, random_state=seed)

In [4]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
print(X.shape)
print(y.shape)

(83578, 1173)
(83578, 1)


In [18]:
epochs = 100
batch_size = 69

In [19]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(200, input_dim=1173, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
model = create_model()

In [21]:
model.fit(X_train, y_train, epochs=epochs)#, batch_size=batch_size)

Epoch 1/100
58504/58504 [==============================] - 2s 40us/step - loss: 0.1277 - acc: 0.9399
Epoch 2/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1116 - acc: 0.9449
Epoch 3/100
58504/58504 [==============================] - 2s 41us/step - loss: 0.1096 - acc: 0.9458
Epoch 4/100
58504/58504 [==============================] - 2s 39us/step - loss: 0.1081 - acc: 0.9452
Epoch 5/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1075 - acc: 0.9458
Epoch 6/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1064 - acc: 0.9473
Epoch 7/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1056 - acc: 0.9478
Epoch 8/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1054 - acc: 0.9477
Epoch 9/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1048 - acc: 0.9478
Epoch 10/100
58504/58504 [==============================] - 2s 37us/step - loss: 0.1044 - a

In [22]:
pred = model.predict(X_cv)

In [23]:
bin_pred = Binarizer(threshold=.5).transform(pred)

In [24]:
f1 = f1_score(y_cv, bin_pred)
f1

0.6594274432379073